In [1]:
import os
import nltk
import math
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.metrics import accuracy_score, f1_score
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Embedding, BatchNormalization, Activation, Input, Add, Concatenate,\
                         Bidirectional, SimpleRNN, LSTM, GRU
from keras_layer_normalization import LayerNormalization
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Embedding, Dropout, BatchNormalization, Activation, Input, \
    Conv1D, MaxPool1D, Flatten, Concatenate, Add, MaxPooling1D,LSTM
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stopwords = set(stopwords.words("english"))
ps = PorterStemmer()

Using TensorFlow backend.


In [2]:
def tokenize(text):
    """
    :param text: a doc with multiple sentences, type: str
    return a word list, 
    
type|: list
    e.g.
    Input: 'Text mining is to identify useful information.'
    Output: ['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
    """
    return nltk.word_tokenize(text)

def stem(tokens):
    """
    :param tokens: a list of tokens, type: list
    return a list of stemmed words, type: list
    e.g.
    Input: ['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
    Output: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
    """
    ### equivalent code
    # results = list()
    # for token in tokens:
    #     results.append(ps.stem(token))
    # return results

    return [ps.stem(token) for token in tokens]

def n_gram(tokens, n=1):
    """
    :param tokens: a list of tokens, type: list
    :param n: the corresponding n-gram, type: int
    return a list of n-gram tokens, type: list
    e.g.
    Input: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.'], 2
    Output: ['text mine', 'mine is', 'is to', 'to identifi', 'identifi use', 'use inform', 'inform .']
    """
    if n == 1:
        return tokens
    else:
        results = list()
        for i in range(len(tokens)-n+1):
            # tokens[i:i+n] will return a sublist from i th to i+n th (i+n th is not included)
            results.append(" ".join(tokens[i:i+n]))
        return results
    
def filter_stopwords(tokens):
    """
    :param tokens: a list of tokens, type: list
    return a list of filtered tokens, type: list
    e.g.
    Input: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
    Output: ['text', 'mine', 'identifi', 'use', 'inform', '.']
    """
    ### equivalent code
    # results = list()
    # for token in tokens:
    #     if token not in stopwords and not token.isnumeric():
    #         results.append(token)
    # return results

    return [token for token in tokens if token not in stopwords and not token.isnumeric()]

In [3]:
df = pd.read_csv("Quoraquestions.csv")
#answer_count_list = non_anonymous_answer_count+anonymous_answer_count
labels = []

for i in range(df.shape[0]):   
    if(df["anonymous_answer_count"][i] > 0):
        labels.append(1)
        
    else:
        labels.append(0)
Counter(labels)
#Construct the labels of questions: 1 means receiving anonymous answers, 0 means not.
xtrain = df["question_title_list"]
x_train = np.array(xtrain)
y_train = np.array(labels)
#x_train contains question texts, y_train contains labels

In [4]:
Counter(labels)

Counter({0: 10028, 1: 1099})

In [5]:
#Preprocessing the question texts

train_texts = x_train
#print(train_texts[0])
train_tokens = [tokenize(text) for text in train_texts]
#print(train_tokens[0])
tokens = []
for text in train_tokens:
    text2 = [i.lower() for i in text]
    text1 = []
    for i in text2:
        if(i != "?"):
            text1.append(i)
    tokens.append(text1)
train_tokens = tokens
#print(train_tokens[0])
filtered_sentence = [filter_stopwords(text) for text in train_tokens]
#print(filtered_sentence[0])
#print(len(train_tokens))
train_stemmed = [stem(tokens) for tokens in filtered_sentence]
#print(train_stemmed[0])
#print(len(train_stemmed))
train_2_gram = [n_gram(tokens, 2) for tokens in train_stemmed]
#print(train_2_gram[0])
train_3_gram = [n_gram(tokens, 3) for tokens in train_stemmed]
#print(train_3_gram[0])
train_feats = list()
for i in range(len(train_texts)):
    train_feats.append(
        train_stemmed[i])


In [6]:
def get_feats_dict(feats, min_freq=-1, max_freq=-1, max_size=-1):
    """
    :param data: a list of features, type: list(list)
    :param min_freq: the lowest fequency that the fequency of a feature smaller than it will be filtered out
    :param max_freq: the highest fequency that the fequency of a feature larger than it will be filtered out
    return a feature dict that maps features to indices, sorted by frequencies
    # Counter document: https://docs.python.org/3.6/library/collections.html#collections.Counter
    """
    # count all features
    feat_cnt = Counter(feats) # ["text", "text", "mine"] --> {"text": 2, "mine": 1}
    if max_size > 0 and min_freq == -1 and max_freq == -1:
        valid_feats = [f for f, cnt in feat_cnt.most_common(max_size)]
    else:
        valid_feats = list()
        for f, cnt in feat_cnt.most_common():
            if (min_freq == -1 or cnt >= min_freq) and \
                (max_freq == -1 or cnt <= max_freq):
                valid_feats.append(f)
    if max_size > 0 and len(valid_feats) > max_size:
        valid_feats = valid_feats[:max_size]        
    print("Size of features:", len(valid_feats))
    
    # build a mapping from features to indices
    feats_dict = dict(zip(valid_feats, range(len(valid_feats))))
    return feats_dict

def get_idf_dict(feats_list):
    """
    :param feats_list: a list of lists of features, type: list(list)
    return an idf vector,
    """
    N = len(feats_list)
    df_dict = Counter()
    for feats in feats_list:
        df_dict.update(set(feats))
    # IDF: log(1 + N/n)
    idf_dict = {f: math.log2(1+N/cnt) for f, cnt in df_dict.items()}
    
    return idf_dict

def get_tfidf_vector(feats, feats_dict, idf_dict):
    """
    :param feats: a list of features, type: list
    :param feats_dict: a dict from features to indices, type: dict
    :param idf_dict: a dict from features to idf, type: dict
    return a feature vector,
    """
    # initialize the vector as all zeros
    vector = np.zeros(len(feats_dict), dtype=np.float)
    # TF: 1 + log(f)
    tf_dict = {f: 1+math.log2(cnt) for f, cnt in Counter(feats).items()}
    for f in tf_dict:
        # get the feature index, return -1 if the feature is not existed
        f_idx = feats_dict.get(f, -1)
        if f_idx != -1:
            tf = tf_dict[f]
            idf = idf_dict[f]
            # set the corresponding element as tf*idf
            vector[f_idx] = tf*idf
    return vector

In [7]:
feats_dict = get_feats_dict(
    chain.from_iterable(train_feats))

idf_dict = get_idf_dict(train_feats)
train_tfidf_feats_matrix = np.vstack([
    get_tfidf_vector(f, feats_dict, idf_dict) for f in train_feats])

Size of features: 10703


In [8]:
from imblearn.over_sampling import SMOTE, ADASYN

#Using smote to solve the unbalanced data problems
X_resampled_smote, y_resampled_smote = SMOTE().fit_sample(train_tfidf_feats_matrix,y_train)
print(len(y_resampled_smote))
Counter(y_resampled_smote)
x_train = X_resampled_smote
y_train = y_resampled_smote
data_num= x_train.shape[0]
index = np.arange(data_num)  # 生成下标  
np.random.shuffle(index)
print(index)
x_train = x_train[index]
y_train = y_train[index]

20056
[18826 16377 16172 ...   854  3766 12553]


In [9]:
from sklearn.metrics import roc_auc_score
def computeAUC(y_true,y_score):
    auc = roc_auc_score(y_true,y_score)
    print("auc = ",auc)
    return auc

In [10]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train,y_train, random_state=0)


/Users/macbook/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
#Using logistic regression to evaluate the classification model.
from sklearn.linear_model.logistic import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)
predicted_probs_train = lr.predict_proba(x_train)
predicted_probs_train = [x[1] for  x in predicted_probs_train]
computeAUC(y_train, predicted_probs_train)

predicted_probs_test_new = lr.predict_proba(x_test)
predicted_probs_test_new = [x[1] for x in predicted_probs_test_new]
computeAUC(y_test, predicted_probs_test_new)


auc =  0.9971840163841104
auc =  0.92137218954498


0.92137218954498

In [12]:
#Using the RandomForest Model to evaluate the classification model
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100,
                                oob_score= True,
                                min_samples_split=2,
                                min_samples_leaf=50,
                                n_jobs=-1,
                                class_weight='balanced_subsample',
                                bootstrap=True)

rf.fit(x_train, y_train)

predicted_probs_train = rf.predict_proba(x_train)
predicted_probs_train = [x[1] for x in predicted_probs_train]
computeAUC(y_train, predicted_probs_train)
#使用训练的模型来预测test_new数据（validataion data）
predicted_probs_test_new = rf.predict_proba(x_test)
predicted_probs_test_new = [x[1] for x in predicted_probs_test_new]
computeAUC(y_test, predicted_probs_test_new)

/Users/macbook/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


auc =  0.9341712262158213
auc =  0.9341908722277392


0.9341908722277392